In [1]:
from functions_Chebyshev_basis import g_D_symbolic_coefficients_dict
from functions_Chebyshev_basis import restore_matrices
from functions_Chebyshev_basis import Augmented_Lagrangian
from functions_Chebyshev_basis import update_Lagrangian_coefficients
from functions_Chebyshev_basis import generate_M_d
import numpy as np
from scipy.integrate import quad
from functools import partial
import jax
import jax.numpy as jnp
from scipy.optimize import minimize
import sympy as sp

In [35]:
D = 2
d = 8
L = 4
rho = 10

In [36]:
expanded_result, coefficients_dict,polynomial,x = g_D_symbolic_coefficients_dict(D)
print("Expanded Result:")
print(expanded_result)
print("\nCoefficients Dictionary:")
for vars_tuple, coeff in coefficients_dict.items():
    print(f"Coefficient of {vars_tuple}: {coeff}")

orders_list = list(coefficients_dict.keys())
coefficients_list = list(coefficients_dict.values())
print(orders_list)
print(coefficients_list)

Expanded Result:
-x1**8*x2**8 + 0.5*x1**2 + 0.5*x2**2

Coefficients Dictionary:
Coefficient of (8, 8): -1.0
Coefficient of (2, 0): 0.5
Coefficient of (0, 2): 0.5
[(8, 8), (2, 0), (0, 2)]
[-1.0, 0.5, 0.5]


In [37]:
x = sp.Symbol('x')

def chebyshev_polynomial(n):
    return 1/2*sp.chebyshevt(n, x)

def definite_integrate_chebyshev(n, a, b):
    T_n = chebyshev_polynomial(n)
    definite_integral = sp.integrate(T_n, (x, a, b))
    return definite_integral

In [38]:
# This block is for the matrix initialization and flatten as x_input

x_mu_D_L_list = [[] for _ in range(D)]
x_R_L_list = [[] for _ in range(D)]

# 9/28 update, we first generate a list of different moments of measure, then in the augumented lagrangian, we can generate the M_d matrix from our list of moments of measure
# This can fix the problem that when we try to differentiate the ϕn(µ), it actully differentiate the list of moments of measure not the M_d matrix 

# Define the mu_list
list_size = 2*d+3
my_list = np.array([definite_integrate_chebyshev(i,-1, 1) for i in range(2*d+3)])
del x
# # generate the list of list of both matrix
for l in range(L):
    for i in range(D):
        x_mu_D_L_list[i].append(my_list)
        x_R_L_list[i].append(np.random.uniform(-1, 1, size=(d+1, d+1)))

x_matrix_list_new = x_mu_D_L_list+x_R_L_list
x_input = np.array([])
for matrix_index in range(len(x_matrix_list_new)):
    for l in range(L):
        flattened_array = x_matrix_list_new[matrix_index][l].flatten()
        x_input = np.concatenate((x_input, flattened_array))

print(len(x_input))




800


In [39]:
# create la
Lagrangian_coefficient = [[[] for _ in range(D)],[[] for _ in range(D)],[[] for _ in range(D)]]

for l in range(L):
    for i in range(D):
        Lagrangian_coefficient[0][i].append(np.zeros((d+1,d+1)))

for l in range(L):
    for i in range(D):
        Lagrangian_coefficient[1][i].append(0)


for l in range(L):
    for i in range(D):
        Lagrangian_coefficient[2][i].append(np.zeros(2*d+3))


print(Lagrangian_coefficient[1])


[[0, 0, 0, 0], [0, 0, 0, 0]]


In [40]:
# Initialize the lagrangian coefficient as 0
Lagrangian_coefficient = 0
rho = 100

In [42]:
#implement the augumented lagrangian
aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                 coefficients_list=coefficients_list,
                                 Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)

#use jax autogradient to generate the gradient
aug_lagrangian_partial_gradient = jax.grad(aug_lagrangian_partial)
update_Lagrangian_coefficients(d,D,L,x_input,Lagrangian_coefficient,rho)

-16821.5840742018

In [162]:
print(aug_lagrangian_partial.keywords['Lagrangian_coefficient'])
print(aug_lagrangian_partial.keywords['rho'])

0
100


In [163]:
# iteration = 0
# while True:
#     iteration += 1
aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                coefficients_list=coefficients_list,
                                Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)
aug_lagrangian_partial_gradient = jax.grad(aug_lagrangian_partial)

result = minimize(aug_lagrangian_partial, x0=x_input,
                method='L-BFGS-B',
                jac=aug_lagrangian_partial_gradient,
                options={
                    'gtol': 1e-3,             # Stopping criterion (relative gradient)
                    'ftol': 1e-4,             # Stopping criterion (absolute value)
                })

print("-"*40)
# print("This is {} iteration".format(iteration))

print("Minimum value of the Augmented Lagrangian function:", result.fun)
print("Was the optimization successful?", result.success)
print("Number of iterations:", result.nit)
print(result.message)

Lagrangian_coefficient = update_Lagrangian_coefficients(d,D,L,result.x,Lagrangian_coefficient,rho)
print("-"*40)

x_input = result.x
print("Lagrangian_coefficient:",Lagrangian_coefficient)
print("rho:",rho)

x_mu_D_L_list,_= restore_matrices(x_input,d,D,L)
print(x_mu_D_L_list)


----------------------------------------
Minimum value of the Augmented Lagrangian function: 0.00939176231622696
Was the optimization successful? True
Number of iterations: 280
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
----------------------------------------
Lagrangian_coefficient: -2.224901721219602
rho: 100
[[array([ 0.05738336,  0.00457987,  0.00155813, -0.0026481 ,  0.00534707,
       -0.00518775,  0.01141017, -0.01041721,  0.0244057 , -0.02029933,
        0.05306924, -0.03876852,  0.11628455, -0.07230543,  0.25859638,
       -0.13457381,  0.59150539,  0.        , -0.00309598]), array([ 5.17540909e-03, -7.49794432e-05,  7.11065219e-04,  2.27043941e-05,
        6.18180306e-03,  3.80650205e-05,  1.24080172e-02,  2.25014832e-04,
        2.76356384e-02,  7.53586417e-04,  6.07465272e-02,  2.37654685e-03,
        1.33537762e-01,  6.86142577e-03,  2.93889915e-01,  1.84772000e-02,
        6.46450118e-01,  0.00000000e+00, -3.09597524e-03]), array([ 4.93515194e-03, -3.17774186e-04,  1

In [195]:
result

-1.9877057870229085

In [187]:
x_input = x_input - 0.1*aug_lagrangian_partial_gradient(x_input)
print(x_input)

[ 0.05387476 -0.00633094 -0.01774413 ...  0.27339664 -0.23948748
 -0.31913447]


In [188]:
x_mu_D_L_list,_= restore_matrices(x_input,d,D,L)
print(x_mu_D_L_list)

[[Array([ 0.05387476, -0.00633094, -0.01774413, -0.00324622,  0.01044292,
       -0.01343435,  0.02212571, -0.03159237,  0.07270647, -0.05026062,
        0.05963268, -0.04498215,  0.11794849, -0.07381953,  0.25795773,
       -0.13542438,  0.59105   ,  0.        , -0.00309598], dtype=float32), Array([ 0.01033466, -0.00365945, -0.03393409, -0.00070638,  0.01012003,
        0.00205528,  0.02086094,  0.00227419,  0.0797924 ,  0.00243714,
        0.06598138,  0.00366161,  0.13744009,  0.00683825,  0.29415873,
        0.01795037,  0.6470992 ,  0.        , -0.00309598], dtype=float32), Array([-2.3817704e-03, -2.3227674e-03, -8.3801858e-03, -4.8644945e-04,
        1.2129983e-02, -9.0765604e-04,  1.1284473e-02, -6.9938088e-03,
       -2.3769358e-02, -1.6791198e-02,  4.7254264e-02, -6.1125746e-03,
        1.2895194e-01, -8.1838220e-03,  2.8829789e-01, -1.6969973e-02,
        6.3739771e-01,  0.0000000e+00, -3.0959751e-03], dtype=float32), Array([ 0.1016996 , -0.00448303, -0.0150631 ,  0.00346313,

In [189]:
aug_lagrangian_partial_gradient(x_input)

Array([-0.34379572, -2.3939903 , -5.681319  , ...,  1.783972  ,
        0.8618355 ,  0.75519305], dtype=float32)

In [190]:
x_M_D_L_list = generate_M_d(x_mu_D_L_list,d,D,L)
l_product_list = []
for l in range(L):
    moment_product = 1
    for i in range(D):
        moment_product *= x_M_D_L_list[i][l][0,0]
    l_product_list.append(moment_product)

max_index = l_product_list.index(max(l_product_list))
x_output = [x_mu_D_L_list[i][max_index][1] for i in range(D)]
print(x_output)

[Array(-5.9257727e-06, dtype=float32), Array(-0.01607369, dtype=float32), Array(0.006082, dtype=float32), Array(-0.0022346, dtype=float32), Array(0.00391493, dtype=float32), Array(-0.00812851, dtype=float32)]


In [192]:
# Define the function based on the given equation
def compute_function(x):
    D = len(x)
    sum_term = (1 / D) * np.sum(2 * x**2 - 1)
    product_term = np.prod(np.cos(8 * np.arccos(x)))
    result = sum_term - product_term
    return result

x = np.array(x_output)
result_out = compute_function(x)
print(result_out)
print((result_out+2)/2)


-1.9877057870229085
0.006147106488545773
